In [1]:
# Core
import numpy as np
import pandas as pd
import dask.dataframe as dd
import dask.multiprocessing

import glob

# Astronomy
import rebound

In [2]:
# Local imports
import kepler_sieve
from horizons import make_sim_horizons
from rebound_sim import make_sim_planets
from rebound_integrate import integrate_numpy, integrate_df, integrate_mjds
from planets_test import get_integration_diff
from db_config import db_engine
from db_utils import sp2df, get_columns, df2csv, csv2db, csvs2db

In [7]:
# Typing
from typing import Tuple

In [47]:
# Make sim with planets
epoch = 59000
sim_epoch = make_sim_planets(epoch=epoch)

In [9]:
# Date range to process
mjd0: int = epoch-100
mjd1: int = epoch+100

In [10]:
# Shared time_step and save_step
steps_per_day: int = 1

# Flags for building simulation archive
save_elements: bool = True
progbar: bool = True

In [11]:
# Save simulation as Numpy arrays
# body_ids, body_names, epochs, q, v, elts = integrate_numpy(sim_epoch=sim_epoch, mjd0=mjd0, mjd1=mjd1, time_step=time_step, save_elements=save_elements, progbar=True)

In [16]:
mjds = np.arange(58900, 59101)
body_ids, body_names, q, v, elts = integrate_mjds(sim_epoch=sim_epoch, mjds=mjds, save_elements=True, progbar=True)

epoch=59000.0


  0%|          | 0/101 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [17]:
df = integrate_df(sim_epoch=sim_epoch, mjd0=mjd0, mjd1=mjd1, interval_p=1, interval_q=1, 
                  save_elements=save_elements, progbar=True)

epoch=59000.0


  0%|          | 0/3651 [00:00<?, ?it/s]

  0%|          | 0/3650 [00:00<?, ?it/s]

In [18]:
df[0:10]

,TimeID,BodyID,MJD,qx,qy,qz,vx,vy,vz,a,e,inc,Omega,omega,f,M
0,79704000,10,55350.0,-0.004091,0.002036,0.000013,-0.000002,-0.000006,3.848410e-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,79704000,1,55350.0,0.316987,-0.240510,-0.049266,0.011411,0.023751,8.938388e-04,0.387099,0.205626,0.122249,0.843300,0.508838,-1.999692,-1.602653
2,79704000,2,55350.0,-0.675194,0.256483,0.042228,-0.007267,-0.019013,1.590128e-04,0.723330,0.006810,0.059245,1.337804,0.958506,0.483104,0.476806
3,79704000,399,55350.0,-0.314337,-0.963601,0.000034,0.016094,-0.005338,-3.727066e-07,0.999690,0.017134,0.000033,1.982947,-0.177364,2.595936,2.577955
4,79704000,301,55350.0,-0.312236,-0.965290,0.000190,0.016447,-0.004905,3.947402e-05,0.002546,0.061584,0.091279,-1.359143,-2.299887,2.984025,2.963779
5,79704000,4,55350.0,-1.644921,-0.121043,0.037724,0.001567,-0.012766,-3.058587e-04,1.523755,0.093306,0.032269,0.864364,-1.281375,3.633213,-2.556157
6,79704000,5,55350.0,4.861469,-1.018005,-0.104634,0.001455,0.007745,-6.472818e-05,5.202790,0.048901,0.022756,1.754244,-1.501117,-0.459688,-0.417686
7,79704000,6,55350.0,-9.498782,-0.593382,0.388313,0.000049,-0.005580,9.511699e-05,9.519125,0.054167,0.043422,1.983303,-0.407909,1.629130,1.520777
8,79704000,7,55350.0,20.065736,-0.957272,-0.263523,0.000159,0.003745,1.186254e-05,19.271609,0.043002,0.013473,1.292190,1.687635,-3.027608,-3.017503
9,79704000,8,55350.0,25.077337,-16.492668,-0.238297,0.001704,0.002641,-9.366565e-05,30.230740,0.011491,0.030859,2.299768,-1.952692,-0.928689,-0.910379


In [15]:
mjd0 = 55350
mjd1 = 62650
body_collection = 'Planets'
dfd = get_integration_diff(body_collection='Planets', mjd0=mjd0, mjd1=mjd1, by_date=False)

In [16]:
dfd

,TimeID,BodyID,MJD,dq,dv,dq_rel,dv_rel
0,79704000,1,55350.0,1.266140e-05,8.906567e-07,3.323542e-05,3.221320e-05
1,79704000,2,55350.0,6.010629e-06,1.690565e-07,8.309523e-06,8.358312e-06
2,79704000,4,55350.0,2.133959e-06,1.914419e-08,1.404944e-06,1.373448e-06
3,79704000,5,55350.0,2.155405e-07,3.161788e-10,4.152260e-08,4.195980e-08
4,79704000,6,55350.0,2.329974e-08,1.299742e-11,2.450806e-09,2.331962e-09
...,...,...,...,...,...,...,...
80306,90216000,8,62650.0,1.040801e-09,7.188373e-13,3.938465e-11,2.601129e-10
80307,90216000,9,62650.0,8.755010e-10,4.939106e-13,3.034195e-11,2.166224e-10
80308,90216000,10,62650.0,1.194718e-08,4.332699e-12,2.010679e-06,5.891571e-07
80309,90216000,301,62650.0,4.599636e-06,3.356210e-07,4.599980e-06,1.949895e-05


In [22]:
params={'BodyCollectionName': 'Planets', 'epoch': 59100}
df_h = sp2df('JPL.GetHorizonsStateCollection', params)

In [41]:
df_k = integrate_df(sim_epoch=sim_epoch, mjd0=58900, mjd1=59100, interval_p=1, interval_q=1, 
                    save_elements=save_elements, progbar=True)

epoch=59000.0


  0%|          | 0/101 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [42]:
mask = (df_k.TimeID == 59000*24*60)
df_k1 = df_k[mask][['BodyID', 'qx', 'qy', 'qz']]
df_k1

,BodyID,qx,qy,qz
1100,10,-0.005040,0.007027,0.000058
1101,1,-0.398018,-0.035904,0.032599
1102,2,-0.284042,-0.662575,0.006970
1103,399,-0.356169,-0.944135,0.000100
1104,301,-0.358599,-0.943750,0.000324
1105,4,0.608101,-1.261972,-0.041575
1106,5,1.620732,-4.902980,-0.015923
1107,6,4.512971,-8.933589,-0.024328
1108,7,15.869135,11.839889,-0.161613
1109,8,29.332922,-5.892970,-0.554652


In [ ]:
def test_integration_df(df: pd.DataFrame, epoch: int):
    """Test an integration data frame against Horizons"""
    # Mask the data frame to match the desired epoch
    mask = (df_k.TimeID == epoch*24*60)
    # Extract this date and the fields to test
    df_k = df[mask][['BodyID', 'qx', 'qy', 'qz', 'vx', 'vy', 'vz']]
    
    # Look up the configuration on Horizons
    params={'BodyCollectionName': 'Planets', 'epoch': epoch}
    df_h = sp2df('JPL.GetHorizonsStateCollection', params)
    # Filter down to just the relevant columns
    df_h[['BodyID', 'qx', 'qy', 'qz', 'vx', 'vy', 'vz']]

In [40]:
df_h = df_h[['BodyID', 'qx', 'qy', 'qz', 'vx', 'vy', 'vz']]
df_h

,BodyID,qx,qy,qz,vx,vy,vz
0,10,-0.005826,0.006608,0.000081,-0.000008,-0.000005,2.220491e-07
1,1,-0.317875,-0.317046,0.002258,0.014525,-0.018247,-2.823499e-03
2,2,0.494811,0.527669,-0.021659,-0.014655,0.013922,1.036568e-03
3,399,0.969669,-0.245238,0.000091,0.004018,0.016585,-1.055631e-06
4,301,0.971436,-0.243200,-0.000048,0.003589,0.016942,3.932465e-05
5,4,1.383769,-0.026631,-0.034705,0.000864,0.015180,2.970566e-04
6,5,2.307919,-4.578456,-0.032642,0.006645,0.003755,-1.642408e-04
7,6,4.973013,-8.670475,-0.047221,0.004529,0.002761,-2.283492e-04
8,7,15.628033,12.134521,-0.157395,-0.002441,0.002923,4.248123e-05
9,8,29.391047,-5.583021,-0.562374,0.000565,0.003103,-7.691335e-05


In [50]:
sim_epoch.particles['Earth']

<rebound.Particle object, m=3.003489614531686e-06 x=-0.3561687722518891 y=-0.9441353157289663 z=9.99339693475901e-05 vx=0.01585208637634398 vy=-0.00601929192652646 vz=4.071973103165173e-07>